https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/hela-sverige-ska-fungera--kristdemokraternas_H9024221

In [2]:
import pandas as pd

In [3]:
# currency multiplier used in this budget
multiplier = 1_000

In [4]:
html_url = ('https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/'
            'hela-sverige-ska-fungera--kristdemokraternas_H9024221/html#_Toc84341143')

In [5]:
tables = pd.read_html(html_url)

In [6]:
len(tables)

15

In [7]:
df = tables[10]

In [8]:
df.columns = df.iloc[0]
df = df.iloc[1:]

In [9]:
df = df.replace('±0', '0')
df = df.dropna(how='any')

In [10]:
numeric_cols = ['Regeringens förslag', 'Avvikelse från regeringen']

for numeric_col in numeric_cols:
    df[numeric_col] = df[numeric_col].str.replace('−', '-')
    df[numeric_col] = df[numeric_col].astype(int)
    df[numeric_col] = df[numeric_col] * multiplier

In [12]:
df.columns = ['Utgiftsområde', 'Namn', 'Regeringens förslag', 'Avvikelse från regeringen']

In [13]:
df.to_csv('../data/budgetmotion-2022-kd.csv', index=False)